## Flexibility and Shedding
By default, an episode ends if one of the following conditions is met:
1. **End of Chronics**: The agent reaches the end of the chronics (timeseries) for an Episode (no more data => scenario ends)
2. **Disconnected Generator / Load**: A load, generator is disconnected from the grid. This also applies to Energy Storage Systems (ESSs) if they are being controlled (agent told it to charge / discharge some energy)
3. **Islanding**: The grid splits into multiple parts / islands. For example, a load of generator is alone on a busbar. 
4. **Powerflow Divergence**: The redispatching routine "diverges": this means that Grid2OP is not able to find a setpoint for the generators for which both "pmin, pmax and the ramps constrained" as well as the "load / generator balancing the solver "diverges" and is not able to compute the next state (often meaning there are no feasible solutions to the Kirchoff's laws)

This notebook explores what happens if condition (2) is loosened, that is if we allow loads and generators to become disconnected from the grid. As a bonus we also examine the possibility to redispatch flexible loads. 

### Flexibility
Flexibility is the ability to perform demand response at loads. That is flexibility allows loads to be controlled. The additional control this offers may allow for a wider diveresity of scenarios to successfully be resolved. <br>
Currently a load is either flexible or not. If it is flexible, the demand can be changed anywhere between 0 and 100% of the load's maximum size (subject to ramping constraints). In practise, commercial or residential participants in demand response may only have partial flexibility.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import grid2op
from grid2op.PlotGrid import PlotMatplot
from pathlib import Path
env_dir = Path.cwd().parent / "grid2op" / "data_test" / "5bus_example_with_flexibility"
env = grid2op.make(env_dir, support_shedding=False)
plotter = PlotMatplot(env.observation_space, gen_name=True, line_id=True, load_name=True)
obs = env.reset()
plotter.plot_obs(obs)
plt.show()

Here we simulate a single flexibility action on the load at substation 3. <br> Since the flexibility requested results in a larger demand than the size of the load, it is truncated.

In [ ]:
import ipywidgets as widgets

tab = widgets.Tab(children=[out1 := widgets.Output(), out2 := widgets.Output(), out3 := widgets.Output()])
tab.set_title(0, "Initial"); tab.set_title(1, "Flex Active"); tab.set_title(2, "Step After Flex")
display(tab)

env.set_id(0)
obs = env.reset()
nothing_obs, reward, done, info = env.step(env.action_space({}))

env.set_id(0)
obs = env.reset()
flex_act = env.action_space({"flexibility":{"load_3_1":2.0}}) # Above maximum (9.7), so will be truncated!
                             # "redispatch":{"gen_1_1":0.1}})
flex_obs, reward, done, flex_info = env.step(flex_act)
next_obs, reward, done, next_info = env.step(env.action_space({}))

figsize = (8,6)
with out1:
    fig1, axes1 = plt.subplots(nrows=1, ncols=1, figsize=figsize)
    plotter.plot_obs(nothing_obs, figure=fig1)
    plt.show(fig1)
with out2:
    fig2, axes2 = plt.subplots(nrows=1, ncols=1, figsize=figsize)
    plotter.plot_obs(flex_obs, figure=fig2)
    plt.show(fig2)
with out3:
    fig3, axes3 = plt.subplots(nrows=1, ncols=1, figsize=figsize)
    plotter.plot_obs(next_obs, figure=fig3)
    plt.show(fig3)